# **GPT Settings**

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.9.0
    Uninstalling openai-2.9.0:
      Successfully uninstalled openai-2.9.0


In [ ]:
import openai
import os

# **Extract Hex dump Output**

In [ ]:
with open('/content/handshake_hex.txt', 'r') as f:  # Adjust path if needed
    hex_dump = f.read()

**Producing clean hex:**

In [ ]:
import re
# Extract hex bytes using regex
hex_lines = []
for line in hex_dump.strip().split('\n'):
    # Extract only valid 2-digit hex groups
    hex_bytes = re.findall(r'\b[0-9a-fA-F]{2}\b', line)
    if hex_bytes:
        hex_lines.append(' '.join(hex_bytes))

# Join all hex lines into one full hex stream
clean_hex_output = '\n'.join(hex_lines)
print(clean_hex_output)

02 00 00 00 45 00 00 34 6d 36 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 ca 3b 30 39 38 25 3e 79
00 00 00 00 80 02 ff ff 05 d7 00 00 02 04 ff d7
01 03 03 08 01 01 04 02
02 00 00 00 45 00 00 34 6d 37 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 30 39 ca 3b 57 dc 1f 22 09
38 25 3e 7a 80 12 ff ff 8e c7 00 00 02 04 ff d7
01 03 03 08 01 01 04 02
02 00 00 00 45 00 00 28 6d 38 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 ca 3b 30 39 38 25 3e 7a
57 dc 1f 23 50 10 00 ff c8 bf 00 00
02 00 00 00 45 00 00 35 6d 39 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 ca 3b 30 39 38 25 3e 7a
57 dc 1f 23 50 18 00 ff 58 6a 00 00 48 65 6c 6c
6f 20 53 65 72 76 65 72 21
02 00 00 00 45 00 00 28 6d 3a 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 30 39 ca 3b 57 dc 1f 23 09
38 25 3e 87 50 10 00 ff c8 b2 00 00
02 00 00 00 45 00 00 35 6d 3b 40 00 80 06 00 00
7f 00 00 01 7f 00 00 01 30 39 ca 3b 57 dc 1f 23 09
38 25 3e 87 50 18 00 ff 68 65 00 00 48 65 6c 6c
6f 20 43 6c 69 65 6e 74 21
02 00 00 00 45 00 00 28 6d 3c 40 00 80 06 00 00
7

**Getting rid of spaces from HEX data**

In [ ]:
import pandas as pd
from google.colab import files

# 1. Paste your full hex dump inside the triple quotes
raw_hex = clean_hex_output

# 2. Clean lines and split into packets
lines = [l.strip() for l in raw_hex.splitlines() if l.strip()]

packets_with_spaces = []
current = []

for line in lines:
    # New packet starts when we see the loopback+IPv4 header pattern
    if line.startswith("02 00 00 00 45 00"):
        if current:
            packets_with_spaces.append(" ".join(current))
            current = []
    current.append(line)

# Add the last packet
if current:
    packets_with_spaces.append(" ".join(current))

# 3. Also create a version with no spaces (continuous hex string)
packets_clean = [p.replace(" ", "") for p in packets_with_spaces]

# 4. Build DataFrame
df = pd.DataFrame({
    "packet_id": range(1, len(packets_clean) + 1),
    "hex_with_spaces": packets_with_spaces,
    "hex_clean": packets_clean,
})

# 5. Save to CSV in Colab
csv_name = "tcp_packets_hex.csv"
df.to_csv(csv_name, index=False)
print(f"Saved {len(df)} packets to {csv_name}")

# 6. Offer CSV for download
files.download(csv_name)


Saved 221 packets to tcp_packets_hex.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Loading Wireshark (non hex) output from CMD**

In [ ]:
import pandas as pd

col_names = [
    "packet_id",
    "src_port",
    "dst_port",
    "seq",
    "ack",
    "tcp_hdr_len",
    "tcp_flags",
    "tcp_flag_syn",
    "tcp_flag_ack",
    "window_size",
    "checksum",
    "urgent_pointer",
    "tcp_options",
    "tcp_payload",
]

df = pd.read_csv("/content/handshake_packets.txt",
                 header=None,
                 names=col_names)

df.head()


,packet_id,src_port,dst_port,seq,ack,tcp_hdr_len,tcp_flags,tcp_flag_syn,tcp_flag_ack,window_size,checksum,urgent_pointer,tcp_options,tcp_payload
0,1,56039,12345,0,0,32,0x0002,True,False,65535,0x953f,0,0204ffd70103030801010402,NaN
1,2,12345,56039,0,1,32,0x0012,True,True,65535,0x6c5e,0,0204ffd70103030801010402,NaN
2,3,56039,12345,1,1,20,0x0010,False,True,255,0xa656,0,NaN,NaN
3,4,56039,12345,1,1,20,0x0018,False,True,255,0x3601,0,NaN,48656c6c6f2053657276657221
4,5,12345,56039,1,14,20,0x0010,False,True,255,0xa649,0,NaN,NaN


**Decoding each TCP flag from flags column**

In [ ]:
def decode_flags(flags_hex):
    # flags_hex like '0x0018'
    flags = int(str(flags_hex), 16)
    return pd.Series({
        "flag_FIN": bool(flags & 0x01),
        "flag_SYN": bool(flags & 0x02),
        "flag_RST": bool(flags & 0x04),
        "flag_PSH": bool(flags & 0x08),
        "flag_ACK": bool(flags & 0x10),
        "flag_URG": bool(flags & 0x20),
        "flag_ECE": bool(flags & 0x40),
        "flag_CWR": bool(flags & 0x80),
    })

df_flags = df["tcp_flags"].apply(decode_flags)
df = pd.concat([df, df_flags], axis=1)

df.head()


,packet_id,src_port,dst_port,seq,ack,tcp_hdr_len,tcp_flags,tcp_flag_syn,tcp_flag_ack,window_size,...,tcp_options,tcp_payload,flag_FIN,flag_SYN,flag_RST,flag_PSH,flag_ACK,flag_URG,flag_ECE,flag_CWR
0,1,56039,12345,0,0,32,0x0002,True,False,65535,...,0204ffd70103030801010402,NaN,False,True,False,False,False,False,False,False
1,2,12345,56039,0,1,32,0x0012,True,True,65535,...,0204ffd70103030801010402,NaN,False,True,False,False,True,False,False,False
2,3,56039,12345,1,1,20,0x0010,False,True,255,...,NaN,NaN,False,False,False,False,True,False,False,False
3,4,56039,12345,1,1,20,0x0018,False,True,255,...,NaN,48656c6c6f2053657276657221,False,False,False,True,True,False,False,False
4,5,12345,56039,1,14,20,0x0010,False,True,255,...,NaN,NaN,False,False,False,False,True,False,False,False


**Categorizing PAcket Types**

In [ ]:
def classify_pkt(row):
    syn = row["flag_SYN"]
    ack = row["flag_ACK"]
    fin = row["flag_FIN"]
    rst = row["flag_RST"]

    if syn and not ack:
        return "SYN"
    if syn and ack:
        return "SYN-ACK"
    if fin and ack:
        return "FIN-ACK"
    if fin and not ack:
        return "FIN"
    if rst:
        return "RST"
    if ack:
        return "ACK"
    return "OTHER"

df["pkt_type"] = df.apply(classify_pkt, axis=1)

df[["packet_id", "src_port", "dst_port", "tcp_flags", "pkt_type"]].head(20)


,packet_id,src_port,dst_port,tcp_flags,pkt_type
0,1,56039,12345,0x0002,SYN
1,2,12345,56039,0x0012,SYN-ACK
2,3,56039,12345,0x0010,ACK
3,4,56039,12345,0x0018,ACK
4,5,12345,56039,0x0010,ACK
5,6,12345,56039,0x0018,ACK
6,7,56039,12345,0x0010,ACK
7,8,12345,56039,0x0011,FIN-ACK
8,9,56039,12345,0x0010,ACK
9,10,56039,12345,0x0011,FIN-ACK


**Merging Two dataframes for capturing all TCP header fields with Values**

In [ ]:
df_hex = pd.read_csv("/content/tcp_packets_hex.csv")  # packet_id, hex_with_spaces, hex_clean

df_full = df_hex.merge(df, on="packet_id", how="left")

df_full.to_csv("/content/tcp_handshake_full_tcp_headers.csv", index=False)


**Adding Flow Connections**

In [ ]:
import pandas as pd

df = pd.read_csv("/content/tcp_handshake_full_tcp_headers.csv")

# loopback, so we can define a flow just by ports (smallest-biggest)
df["flow_id"] = df.apply(
    lambda r: f"{min(r.src_port, r.dst_port)}-{max(r.src_port, r.dst_port)}",
    axis=1
)
df.head()


,packet_id,hex_with_spaces,hex_clean,src_port,dst_port,seq,ack,tcp_hdr_len,tcp_flags,tcp_flag_syn,...,flag_FIN,flag_SYN,flag_RST,flag_PSH,flag_ACK,flag_URG,flag_ECE,flag_CWR,pkt_type,flow_id
0,1,02 00 00 00 45 00 00 34 6d 36 40 00 80 06 00 0...,02000000450000346d364000800600007f0000017f0000...,56039,12345,0,0,32,0x0002,True,...,False,True,False,False,False,False,False,False,SYN,12345-56039
1,2,02 00 00 00 45 00 00 34 6d 37 40 00 80 06 00 0...,02000000450000346d374000800600007f0000017f0000...,12345,56039,0,1,32,0x0012,True,...,False,True,False,False,True,False,False,False,SYN-ACK,12345-56039
2,3,02 00 00 00 45 00 00 28 6d 38 40 00 80 06 00 0...,02000000450000286d384000800600007f0000017f0000...,56039,12345,1,1,20,0x0010,False,...,False,False,False,False,True,False,False,False,ACK,12345-56039
3,4,02 00 00 00 45 00 00 35 6d 39 40 00 80 06 00 0...,02000000450000356d394000800600007f0000017f0000...,56039,12345,1,1,20,0x0018,False,...,False,False,False,True,True,False,False,False,ACK,12345-56039
4,5,02 00 00 00 45 00 00 28 6d 3a 40 00 80 06 00 0...,02000000450000286d3a4000800600007f0000017f0000...,12345,56039,1,14,20,0x0010,False,...,False,False,False,False,True,False,False,False,ACK,12345-56039


**Getting individual handshake outputs**

In [ ]:
handshakes = []

for flow_id, g in df.groupby("flow_id"):
    g = g.sort_values("packet_id")

    syn = g[g["pkt_type"] == "SYN"].head(1)
    if syn.empty:
        continue

    syn_pkt = syn.iloc[0]

    syn_ack = g[(g["pkt_type"] == "SYN-ACK") &
                (g["packet_id"] > syn_pkt["packet_id"])].head(1)
    if syn_ack.empty:
        continue
    syn_ack_pkt = syn_ack.iloc[0]

    ack = g[(g["pkt_type"] == "ACK") &
            (g["packet_id"] > syn_ack_pkt["packet_id"])].head(1)
    if ack.empty:
        continue
    ack_pkt = ack.iloc[0]

    handshakes.append({
        "flow_id": flow_id,
        "syn_id": int(syn_pkt["packet_id"]),
        "syn_ack_id": int(syn_ack_pkt["packet_id"]),
        "ack_id": int(ack_pkt["packet_id"]),
    })

df_hs = pd.DataFrame(handshakes)
df_hs.head()


,flow_id,syn_id,syn_ack_id,ack_id
0,12345-56039,1,2,3
1,12345-56040,12,13,14
2,12345-56041,23,24,25
3,12345-56042,33,34,35
4,12345-56043,44,45,46


In [ ]:
import pandas as pd

# All packets with headers + pkt_type + hex
df = pd.read_csv("/content/tcp_handshake_full_tcp_headers.csv")

# The handshake summary you just printed
df_hs = pd.DataFrame(handshakes)
  # or re-create from earlier code if needed
print(df_hs.head())


       flow_id  syn_id  syn_ack_id  ack_id
0  12345-56039       1           2       3
1  12345-56040      12          13      14
2  12345-56041      23          24      25
3  12345-56042      33          34      35
4  12345-56043      44          45      46


#  **1. Ground Truth**

In [ ]:
# We'll grab only the fields we need for now
cols_keep = [
    "packet_id",
    "hex_clean",
    "pkt_type",
    "seq",
    "ack",
    "window_size",
    "tcp_flags",
]

df_small = df[cols_keep].set_index("packet_id")

handshake_rows = []

for _, row in df_hs.iterrows():
    syn     = df_small.loc[row["syn_id"]]
    synack  = df_small.loc[row["syn_ack_id"]]
    ackpkt  = df_small.loc[row["ack_id"]]

    handshake_rows.append({
        "flow_id":   row["flow_id"],

        "syn_id":    int(row["syn_id"]),
        "syn_type":  syn["pkt_type"],
        "syn_seq":   syn["seq"],
        "syn_ack":   syn["ack"],
        "syn_wss":   syn["window_size"],
        "syn_flags": syn["tcp_flags"],
        "syn_hex":   syn["hex_clean"],

        "synack_id":    int(row["syn_ack_id"]),
        "synack_type":  synack["pkt_type"],
        "synack_seq":   synack["seq"],
        "synack_ack":   synack["ack"],
        "synack_wss":   synack["window_size"],
        "synack_flags": synack["tcp_flags"],
        "synack_hex":   synack["hex_clean"],

        "ack_id":    int(row["ack_id"]),
        "ack_type":  ackpkt["pkt_type"],
        "ack_seq":   ackpkt["seq"],
        "ack_ack":   ackpkt["ack"],
        "ack_wss":   ackpkt["window_size"],
        "ack_flags": ackpkt["tcp_flags"],
        "ack_hex":   ackpkt["hex_clean"],
    })

df_handshake = pd.DataFrame(handshake_rows)
df_handshake.head()


,flow_id,syn_id,syn_type,syn_seq,syn_ack,syn_wss,syn_flags,syn_hex,synack_id,synack_type,...,synack_wss,synack_flags,synack_hex,ack_id,ack_type,ack_seq,ack_ack,ack_wss,ack_flags,ack_hex
0,12345-56039,1,SYN,0,0,65535,0x0002,02000000450000346d364000800600007f0000017f0000...,2,SYN-ACK,...,65535,0x0012,02000000450000346d374000800600007f0000017f0000...,3,ACK,1,1,255,0x0010,02000000450000286d384000800600007f0000017f0000...
1,12345-56040,12,SYN,0,0,65535,0x0002,02000000450000346d414000800600007f0000017f0000...,13,SYN-ACK,...,65535,0x0012,02000000450000346d424000800600007f0000017f0000...,14,ACK,1,1,255,0x0010,02000000450000286d434000800600007f0000017f0000...
2,12345-56041,23,SYN,0,0,65535,0x0002,02000000450000346d4c4000800600007f0000017f0000...,24,SYN-ACK,...,65535,0x0012,02000000450000346d4d4000800600007f0000017f0000...,25,ACK,1,1,255,0x0010,02000000450000286d4e4000800600007f0000017f0000...
3,12345-56042,33,SYN,0,0,65535,0x0002,02000000450000286d564000800600007f0000017f0000...,34,SYN-ACK,...,65535,0x0012,02000000450000346d574000800600007f0000017f0000...,35,ACK,1,1,255,0x0010,02000000450000346d584000800600007f0000017f0000...
4,12345-56043,44,SYN,0,0,65535,0x0002,02000000450000286d614000800600007f0000017f0000...,45,SYN-ACK,...,65535,0x0012,02000000450000346d624000800600007f0000017f0000...,46,ACK,1,1,255,0x0010,02000000450000346d634000800600007f0000017f0000...


In [ ]:
df_handshake.to_csv("/content/tcp_handshakes_ground_truth.csv", index=False)
print("Saved tcp_handshakes_ground_truth.csv")


Saved tcp_handshakes_ground_truth.csv


In [ ]:
import pandas as pd
import random

df_gt = pd.read_csv("/content/tcp_handshakes_ground_truth.csv")
df_gt.head()


,flow_id,syn_id,syn_type,syn_seq,syn_ack,syn_wss,syn_flags,syn_hex,synack_id,synack_type,...,synack_wss,synack_flags,synack_hex,ack_id,ack_type,ack_seq,ack_ack,ack_wss,ack_flags,ack_hex
0,12345-56039,1,SYN,0,0,65535,0x0002,02000000450000346d364000800600007f0000017f0000...,2,SYN-ACK,...,65535,0x0012,02000000450000346d374000800600007f0000017f0000...,3,ACK,1,1,255,0x0010,02000000450000286d384000800600007f0000017f0000...
1,12345-56040,12,SYN,0,0,65535,0x0002,02000000450000346d414000800600007f0000017f0000...,13,SYN-ACK,...,65535,0x0012,02000000450000346d424000800600007f0000017f0000...,14,ACK,1,1,255,0x0010,02000000450000286d434000800600007f0000017f0000...
2,12345-56041,23,SYN,0,0,65535,0x0002,02000000450000346d4c4000800600007f0000017f0000...,24,SYN-ACK,...,65535,0x0012,02000000450000346d4d4000800600007f0000017f0000...,25,ACK,1,1,255,0x0010,02000000450000286d4e4000800600007f0000017f0000...
3,12345-56042,33,SYN,0,0,65535,0x0002,02000000450000286d564000800600007f0000017f0000...,34,SYN-ACK,...,65535,0x0012,02000000450000346d574000800600007f0000017f0000...,35,ACK,1,1,255,0x0010,02000000450000346d584000800600007f0000017f0000...
4,12345-56043,44,SYN,0,0,65535,0x0002,02000000450000286d614000800600007f0000017f0000...,45,SYN-ACK,...,65535,0x0012,02000000450000346d624000800600007f0000017f0000...,46,ACK,1,1,255,0x0010,02000000450000346d634000800600007f0000017f0000...


# **3. Evalutaion**

In [ ]:
import openai
openai.api_key = "sk-......."

In [ ]:
client = openai.api_key

In [57]:
# =========================
# TASK 3 for TCP (OpenAI 0.28)
# Next-packet generation + Task-3 metrics table
# =========================

import os
import re
import json
import time
import pandas as pd
from io import StringIO




# -------------------------
# CONFIG
# -------------------------
MODELS = ["gpt-4.1", "gpt-3.5-turbo"]
PROMPTING = ["baseline", "oneshot"]

# Speed controls (2-minute-ish runs)
MAX_FLOWS = 10          # use 10 flows = 20 prompts per run (SYN->SYNACK and SYNACK->ACK)
N_REPETITIONS = 10       # start with 1; increase later
SLEEP_SEC = 0.1         # small delay to reduce rate-limit bursts

GT_PATH = "/content/tcp_handshakes_ground_truth.csv"  # <- your attached file path

RAW_LOG_CSV = "task3_tcp_raw_log.csv"
RESULTS_CSV = "task3_tcp_results.csv"

# -------------------------
# Helpers
# -------------------------
HEX_RE = re.compile(r"^[0-9a-fA-F]+$")

def clean_hex(s: str) -> str:
    """Return only hex chars, lowercase."""
    if s is None or (isinstance(s, float) and pd.isna(s)):
        return ""
    s = str(s)
    s = re.sub(r"[^0-9a-fA-F]", "", s)
    return s.lower()

def safe_json_loads(text: str):
    """
    Extract the first JSON object from model output and parse it.
    Falls back to None if not parseable.
    """
    if not text:
        return None
    # Remove code fences
    text2 = "\n".join([ln for ln in text.splitlines() if not ln.strip().startswith("```")]).strip()

    # Try direct
    try:
        return json.loads(text2)
    except Exception:
        pass

    # Try to locate {...}
    m = re.search(r"\{.*\}", text2, flags=re.DOTALL)
    if not m:
        return None
    try:
        return json.loads(m.group(0))
    except Exception:
        return None

def call_gpt(prompt: str, model_name: str, max_retries: int = 4) -> str:
    last_err = None
    for attempt in range(1, max_retries + 1):
        try:
            resp = openai.ChatCompletion.create(
                model=model_name,
                temperature=0,
                messages=[
                    {"role": "system", "content": "You output STRICT JSON only."},
                    {"role": "user", "content": prompt},
                ],
            )
            return resp["choices"][0]["message"]["content"]
        except Exception as e:
            last_err = e
            time.sleep(min(2 ** attempt, 10))
    raise last_err

# -------------------------
# Task-3 TCP prompts (next packet generation)
# -------------------------
ONE_SHOT_EXAMPLE = """
Example:

Input packet (current):
"02000000450000346d364000800600007f0000017f000001ca3b303938253e79000000008002ffff05d700000204ffd70103030801010402"

Output JSON:
{
  "next_packet_type": "SYN-ACK",
  "next_hex": "02000000450000346d374000800600007f0000017f0000013039ca3b57dc1f220938253e7a8012ffff8ec700000204ffd70103030801010402"
}
""".strip()

def build_prompt_baseline(server_state: str, current_hex: str) -> str:
    return f"""
You are given ONE TCP packet as a full hex string.

Task: Predict the NEXT TCP packet in the SAME flow (TCP 3-way handshake context).
Server state: {server_state}

Input current packet hex:
{current_hex}

Return STRICT JSON ONLY with exactly these keys:
{{
  "next_packet_type": "SYN|SYN-ACK|ACK|OTHER",
  "next_hex": "<continuous hex string, no spaces>"
}}

Rules:
- next_hex must be ONLY hex characters (0-9a-f).
- Do not add explanations.
""".strip()

def build_prompt_oneshot(server_state: str, current_hex: str) -> str:
    return f"""
You are given ONE TCP packet as a full hex string.

ONE-SHOT FORMAT REFERENCE:
{ONE_SHOT_EXAMPLE}

Task: Predict the NEXT TCP packet in the SAME flow (TCP 3-way handshake context).
Server state: {server_state}

Input current packet hex:
{current_hex}

Return STRICT JSON ONLY with exactly these keys:
{{
  "next_packet_type": "SYN|SYN-ACK|ACK|OTHER",
  "next_hex": "<continuous hex string, no spaces>"
}}

Rules:
- Think step-by-step silently to ensure correctness.
- next_hex must be ONLY hex characters (0-9a-f).
- Do not add explanations.
""".strip()

def build_prompt(server_state: str, current_hex: str, prompting: str) -> str:
    return build_prompt_baseline(server_state, current_hex) if prompting == "baseline" else build_prompt_oneshot(server_state, current_hex)

# -------------------------
# Classification logic for Task 3 metrics
# Successful / Partial / Unsuccessful + Hallucination
# -------------------------
def classify_transition(expected_type: str, expected_hex: str, pred_type: str, pred_hex: str):
    """
    Task-3 style outcomes:
    - hallucination: not parseable, empty, not hex, or too short
    - successful: correct next_packet_type AND pred_hex exactly equals expected_hex
    - partial: correct next_packet_type BUT pred_hex != expected_hex (still valid hex)
    - unsuccessful: wrong next_packet_type (but still valid output)
    """
    expected_hex = clean_hex(expected_hex)
    pred_hex = clean_hex(pred_hex)
    pred_type = (pred_type or "").strip().upper()
    expected_type = (expected_type or "").strip().upper()

    # hallucination checks
    if not pred_type or not pred_hex:
        return "hallucination"
    if not HEX_RE.match(pred_hex):
        return "hallucination"
    if len(pred_hex) < 40:  # too small to be a real frame+headers in your dataset
        return "hallucination"

    # normalize types
    # allow synonyms like "SYNACK" or "SYN, ACK"
    pred_type = pred_type.replace(",", " ").replace("|", " ").replace("-", " ").strip()
    pred_type = " ".join(pred_type.split())

    def norm_t(t):
        t = t.upper().replace(",", " ").replace("|", " ").replace("-", " ").strip()
        t = " ".join(t.split())
        if t == "SYN ACK":
            return "SYN-ACK"
        return t

    pred_type_n = norm_t(pred_type)
    exp_type_n = norm_t(expected_type)

    if pred_type_n == exp_type_n:
        if pred_hex == expected_hex:
            return "successful"
        else:
            return "partial"
    else:
        return "unsuccessful"

# -------------------------
# Load ground truth + build Task3 samples
# -------------------------
gt = pd.read_csv(GT_PATH)

# Build samples:
# State A: after client sends SYN -> expected server SYN-ACK
# State B: after server sends SYN-ACK -> expected client ACK
samples = []
for _, r in gt.head(MAX_FLOWS).iterrows():
    flow_id = r["flow_id"]

    samples.append({
        "flow_id": flow_id,
        "server_state": "SYN_SENT (expect SYN-ACK)",
        "current_hex": r["syn_hex"],
        "expected_next_type": r["synack_type"],
        "expected_next_hex": r["synack_hex"],
    })
    samples.append({
        "flow_id": flow_id,
        "server_state": "SYNACK_SENT (expect ACK)",
        "current_hex": r["synack_hex"],
        "expected_next_type": r["ack_type"],
        "expected_next_hex": r["ack_hex"],
    })

df_samples = pd.DataFrame(samples)
df_samples["current_hex"] = df_samples["current_hex"].apply(clean_hex)
df_samples["expected_next_hex"] = df_samples["expected_next_hex"].apply(clean_hex)

print("Samples:", df_samples.shape)
print(df_samples[["flow_id","server_state","expected_next_type"]].head())

# -------------------------
# Run experiment + write raw log incrementally
# -------------------------
raw_rows = []
if os.path.exists(RAW_LOG_CSV):
    os.remove(RAW_LOG_CSV)

def append_csv(path, rows, header):
    df = pd.DataFrame(rows)
    df.to_csv(path, mode="a", index=False, header=header)

for model in MODELS:
    for prompting in PROMPTING:
        for run_idx in range(1, N_REPETITIONS + 1):
            for i, row in df_samples.iterrows():
                prompt = build_prompt(row["server_state"], row["current_hex"], prompting)

                out_text = ""
                pred_type = ""
                pred_hex = ""
                status = ""
                err = ""

                try:
                    out_text = call_gpt(prompt, model)
                    obj = safe_json_loads(out_text)
                    if obj is None:
                        status = "hallucination"
                        err = "json_parse_failed"
                    else:
                        pred_type = obj.get("next_packet_type", "")
                        pred_hex = obj.get("next_hex", "")
                        status = classify_transition(
                            expected_type=row["expected_next_type"],
                            expected_hex=row["expected_next_hex"],
                            pred_type=pred_type,
                            pred_hex=pred_hex
                        )
                except Exception as e:
                    status = "hallucination"
                    err = str(e)[:250]

                raw_rows.append({
                    "LLM": model,
                    "Prompting": prompting,
                    "Run": run_idx,
                    "flow_id": row["flow_id"],
                    "server_state": row["server_state"],
                    "expected_next_type": row["expected_next_type"],
                    "expected_next_hex": row["expected_next_hex"],
                    "pred_next_type": pred_type,
                    "pred_next_hex": clean_hex(pred_hex),
                    "status": status,
                    "error": err,
                    "raw_output": (out_text[:800] if out_text else ""),
                })

                # write every row immediately (fast + safe)
                append_csv(RAW_LOG_CSV, raw_rows, header=not os.path.exists(RAW_LOG_CSV))
                raw_rows.clear()

                time.sleep(SLEEP_SEC)

print("✅ Raw log saved:", RAW_LOG_CSV)

# -------------------------
# Compute Task-3 table metrics (counts + rates)
# -------------------------
df_raw = pd.read_csv(RAW_LOG_CSV)

def summarize_group(g):
    total = len(g)
    counts = g["status"].value_counts().to_dict()

    succ = counts.get("successful", 0)
    part = counts.get("partial", 0)
    uns  = counts.get("unsuccessful", 0)
    hall = counts.get("hallucination", 0)

    def rate(x): return (x / total) if total else 0.0

    return pd.Series({
        "#Prompts": total,

        "Successful State Transition (#)": succ,
        "Successful State Transition (Rate%)": round(rate(succ) * 100, 2),

        "Partial State Transition (#)": part,
        "Partial State Transition (Rate%)": round(rate(part) * 100, 2),

        "Unsuccessful State Transition (#)": uns,
        "Unsuccessful State Transition (Rate%)": round(rate(uns) * 100, 2),

        "Hallucinations (#)": hall,
        "Hallucinations (Rate%)": round(rate(hall) * 100, 2),
    })




Samples: (20, 5)
       flow_id               server_state expected_next_type
0  12345-56039  SYN_SENT (expect SYN-ACK)            SYN-ACK
1  12345-56039   SYNACK_SENT (expect ACK)                ACK
2  12345-56040  SYN_SENT (expect SYN-ACK)            SYN-ACK
3  12345-56040   SYNACK_SENT (expect ACK)                ACK
4  12345-56041  SYN_SENT (expect SYN-ACK)            SYN-ACK
✅ Raw log saved: task3_tcp_raw_log.csv


# **3. Final Output**

In [58]:
df_results = (
    df_raw
    .groupby(["LLM", "Prompting", "server_state"], as_index=False)
    .apply(summarize_group)
    .reset_index(drop=True)
)

df_results.to_csv(RESULTS_CSV, index=False)
print("✅ Results saved:", RESULTS_CSV)

df_results

✅ Results saved: task3_tcp_results.csv


/tmp/ipython-input-3490957701.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(summarize_group)


,LLM,Prompting,server_state,#Prompts,Successful State Transition (#),Successful State Transition (Rate%),Partial State Transition (#),Partial State Transition (Rate%),Unsuccessful State Transition (#),Unsuccessful State Transition (Rate%),Hallucinations (#),Hallucinations (Rate%)
0,gpt-3.5-turbo,baseline,SYNACK_SENT (expect ACK),100.0,0.0,0.0,63.0,63.0,0.0,0.0,37.0,37.0
1,gpt-3.5-turbo,baseline,SYN_SENT (expect SYN-ACK),100.0,0.0,0.0,98.0,98.0,0.0,0.0,2.0,2.0
2,gpt-3.5-turbo,oneshot,SYNACK_SENT (expect ACK),100.0,0.0,0.0,97.0,97.0,0.0,0.0,3.0,3.0
3,gpt-3.5-turbo,oneshot,SYN_SENT (expect SYN-ACK),100.0,10.0,10.0,90.0,90.0,0.0,0.0,0.0,0.0
4,gpt-4.1,baseline,SYNACK_SENT (expect ACK),100.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0
5,gpt-4.1,baseline,SYN_SENT (expect SYN-ACK),100.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0
6,gpt-4.1,oneshot,SYNACK_SENT (expect ACK),100.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,0.0
7,gpt-4.1,oneshot,SYN_SENT (expect SYN-ACK),100.0,10.0,10.0,80.0,80.0,10.0,10.0,0.0,0.0
